In [11]:
from dotenv import load_dotenv
load_dotenv()

True

In [ ]:
from tavily import TavilyClient
from typing import Dict, Any
from langchain.tools import tool

tavily_client = TavilyClient()

@tool()
def web_search (query: str)-> Dict[str, Any]:
  """Search the web for information"""
  return tavily_client.search(query)


In [13]:
from langchain_mcp_adapters.client import MultiServerMCPClient

client = MultiServerMCPClient(
    {
       "travel_server": {
         "transport": "streamable_http",
         "url": "https://mcp.kiwi.com"
       }
    }
)

In [14]:
tools = await client.get_tools()

print(tools)

[StructuredTool(name='search-flight', description='\n# Search for a flight\n\n## Description\n\nUses the Kiwi API to search for available flights between two locations on a specific date.\n\n## How it works\n\nThe tool will:\n1. Search for matching locations to resolve airport codes\n2. Find available flights for the specified route and date range\n\n## Method\n\nCall this tool whenever a user wants to search for flights, regardless of whether they provided exact airport codes or just city names.\n\nYou should display the returned results in a markdown table format: Group the results by price (those who are the cheapest), duration (those who are the shortest, i.e. have the smallest \'totalDurationInSeconds\') and the rest (those that could still be interesting).\n\nAlways display for each flight in order:\n  - In the 1st column: The departure and arrival airports, including layovers (e.g. "Paris CDG → Barcelona BCN → Lisbon LIS")\n  - In the 2nd column: The departure and arrival dates 

In [15]:
from langchain_community.utilities import SQLDatabase

db = SQLDatabase.from_uri("sqlite:///advanced-agents/resources/Chinook.db")

@tool
def query_playlist_db(query: str) -> str:

    """Query the database for playlist information"""

    try:
        return db.run(query)
    except Exception as e:
        return f"Error querying database: {e}"

# Create State

In [16]:
from langchain.agents import AgentState

class WeddingState(AgentState):
  origin: str
  destination: str
  guest_count: str
  genre: str

# Creating SubAgents

In [ ]:
from langchain.agents import create_agent

flight_agent = create_agent(
  model='gpt-5-nano',
  tools=tools,
  system_prompt="""
  Você é um agent de Viagens. Busca por voos para os locais mais desejados para casamentos.
  Você não esta autorizado a perguntar nenhuma questão a mais.
  Você deve buscar as melhores opções de voos baseados nos seguintes critérios:
  - Preço (menor, classe economica)
  - Duração (menor)
  - Data (periodo do ano em que você acredita que seja melhor para casamento neste local)
  Para tornar as coisas mais simples, busque apenas uma passagem, apenas ida.
  Você pode fazer multiplas buscas iterativamente até encontrar a melhor opção.
  Você não irá receber informações extras, apenas a origem e o destino. É seu trabalho pensar criticamente sobre a melhor opção.
  Uma vez que você encontrou a melhor opção, deixe o usuário saber sua "shortlist" de opções.
  """
)

In [18]:
wedding_avenue_agent = create_agent(
  model="gpt-5-nano",
  tools=[web_search],
  system_prompt="""
  Você é um especialista em encontrar espaços para casamento. Procure por espaços no local desejado, e com a capacidade de convidados desejados.
  Você não esta autorizado a perguntar nenhuma questão a mais.
  Você deve buscar as melhores opções de espaços para casamento baseados nos seguintes critérios:
  - Preço (menor)
  - Capacidade(quantia exata informada)
  - Avaliação (Mais avaliados)
  Você pode fazer multiplas buscas iterativamente até encontrar as melhores opções
  """
)

In [19]:
dj_agent = create_agent(
  model="gpt-5-nano",
  tools=[query_playlist_db],
  system_prompt="""
  Você é um especialista em playlists. Procure no banco de dados e faça a curadoria da playlist perfeita para um casamento com base no genero escolhido.
  Uma vez que você tiver a playlist, calcule o total da duração e custo da playlist. Cada música tem um preço associado.
  Se você encontrar erros quando estiver fazendo a busca no banco de dados, tente concertar eles fazendo alterações na query.
  Não volte de mãos vazias. Continue tentando fazer a query no banco de dados até que encontre a lista de músicas.
  Você pode precisar fazer multiplas buscas iterativamente para encontrar a melhor opção
  """
)

# Main Coordinator Agent

In [26]:
from langchain.tools import ToolRuntime
from langchain.messages import HumanMessage, ToolMessage
from langgraph.types import Command

@tool
async def call_flight_agent(runtime: ToolRuntime)-> str:
  """Travel agent searches for flights to the desired destination wedding location."""
  origin = runtime.state["origin"]
  destination = runtime.state["destination"]
  
  response = await flight_agent.ainvoke({"messages": [HumanMessage(content=f"Find flights from {origin} to {destination}")]})

  return response["messages"][-1].content

@tool
def call_wedding_avenue_agent(runtime: ToolRuntime)-> str:
  """Venue agent chooses the best venue for the given location and capacity."""
  destination = runtime.state["destination"]
  guest_count = runtime.state["guest_count"]
  query = f"Find wedding venus in {destination} for {guest_count} guests"

  response = wedding_avenue_agent.invoke({"messages": [HumanMessage(content=query)]})

  return response["messages"][-1].content

@tool
def call_dj_agent(runtime: ToolRuntime) -> str:
  """DJ agent curates the perfect playlist for the given genre."""
  genre = runtime.state["genre"]
  query = f"Find {genre} tracks for wedding playlist"
  response = dj_agent.invoke({"messages": [HumanMessage(content=query)]})
  return response['messages'][-1].content

@tool
def update_state(origin: str, destination: str, guest_count: str, genre: str, runtime: ToolRuntime) -> str:
  """Update the state when you know all of the values: origin, destination, guest_count, genre"""
  return Command(update={
    "origin": origin, 
    "destination": destination, 
    "guest_count": guest_count, 
    "genre": genre, 
    "messages": [ToolMessage("Successfully updated state", tool_call_id=runtime.tool_call_id)]}
  )

In [27]:
from langchain.agents import create_agent

coordinator = create_agent(
    model="gpt-5-nano",
    tools=[call_flight_agent, call_wedding_avenue_agent, call_dj_agent, update_state],
    state_schema=WeddingState,
    system_prompt="""
    Você é um Coordenador de Planos de casamento. Você delega tarefas para seus especialistas em buscar voos, espaços para casamento e playlists
    Primeiro procure todas as informações que você precisa para atualizar o estado. Depois que estiver pronto, você pode delegar as tarefas.
    Uma vez que você tiver recebido suas respostas, coordene o casamento perfeito para mim.
    """
)


In [28]:
from langchain.messages import HumanMessage

response = await coordinator.ainvoke(
  {
    "messages": [HumanMessage(content="Estou em cuiaba e gostaria de um casamento em são paulo para 100 convidados, genero rock")]
  }
)

In [29]:
from pprint import pprint

pprint(response)

{'destination': 'São Paulo',
 'genre': 'rock',
 'guest_count': '100',
 'messages': [HumanMessage(content='Estou em cuiaba e gostaria de um casamento em são paulo para 100 convidados, genero rock', additional_kwargs={}, response_metadata={}, id='bd74fc2e-857d-4b58-9313-89da4f04a075'),
              AIMessage(content='', additional_kwargs={'refusal': None}, response_metadata={'token_usage': {'completion_tokens': 744, 'prompt_tokens': 319, 'total_tokens': 1063, 'completion_tokens_details': {'accepted_prediction_tokens': 0, 'audio_tokens': 0, 'reasoning_tokens': 704, 'rejected_prediction_tokens': 0}, 'prompt_tokens_details': {'audio_tokens': 0, 'cached_tokens': 0}}, 'model_provider': 'openai', 'model_name': 'gpt-5-nano-2025-08-07', 'system_fingerprint': None, 'id': 'chatcmpl-Cxa5pEYVqlzmDCNYgLMWVJJ9JMFjj', 'service_tier': 'default', 'finish_reason': 'tool_calls', 'logprobs': None}, id='lc_run--019bb7da-cbc8-76c1-b037-8670a2b71df3-0', tool_calls=[{'name': 'update_state', 'args': {'origin': 

In [ ]:
print(response["messages"][-1].content)